In [1]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt

# Change working directory to project root
os.chdir('../../')

import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json
import imageio
from hmr4d.utils.wis3d_utils import make_wis3d
from hmr4d.utils.wis3d_utils import add_joints22_motion_as_lines
from hmr4d.dataset.rich.rich_utils import get_w2az_sahmr, parse_seqname_info
from hmr4d.utils.geo_transform import apply_T_on_points

### Compute mean and std

In [2]:
from omegaconf import OmegaConf
from torch.utils.data import DataLoader, ConcatDataset
from hmr4d.dataset.rich.rich_motion2d import Dataset
yaml_file = "hmr4d/configs/data/motion2d/rich_motion2d.yaml"
data_train_opts = OmegaConf.load(yaml_file).opt.train
dataset = ConcatDataset([Dataset(**dict(opt)) for opt in data_train_opts])
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)

In [3]:
all_bi01_motion2d = []
for batch in tqdm(dataloader, total=len(dataloader)):
    all_bi01_motion2d.append(batch['bi01_motion2d'])
all_bi01_motion2d = torch.cat(all_bi01_motion2d, dim=0)
all_bi01_motion2d.shape

  0%|          | 0/2239 [00:00<?, ?it/s]

100%|██████████| 2239/2239 [00:07<00:00, 295.37it/s]


torch.Size([17906, 120, 22, 2])

In [5]:
all_bi01_motion2d = all_bi01_motion2d.cuda()
mean = all_bi01_motion2d.reshape(-1, 2).mean(dim=0)
std = all_bi01_motion2d.reshape(-1, 2).std(dim=0)
print(mean, std)

tensor([0.4985, 0.4615], device='cuda:0') tensor([0.1516, 0.2168], device='cuda:0')
